# MAC0115 2015 - Professor Routo Terada

## Solução do EP1 em python3
### Por Thales Paiva (<thalespaiva@gmail.com>)

### 0. Enunciado

Este exercício pede que os alunos implementassem algoritmos simples sobre listas. Dada uma lista de números `L`, as seguintes medidas estatísticas de `L` eram esperadas:
1. Máximo
2. Mínimo
3. Média
4. Variância
5. Desvio Padrão

### 1. Máximo

Para calcular o máximo de uma lista, é possível usar a função `max`, já embutida no Python. Porém a ideia do EP era fazer com que o aluno exercitasse a manipulação de listas, iterações, e condicionais. Então preferimos que o aluno calculasse o máximo percorrendo os valores da lista e tomando aquele com maior valor, como abaixo.

In [1]:
def get_max(L):
    max_value = L[0]
    
    for element in L:
        if max_value < element:
            max_value = element
    
    return max_value

Vamos testá-la com uma lista para conferir o resultado:

In [2]:
L = [2.4, 5.1, -4.0, 8.2, -7.5, 3.1, 0.9, 1.8, 6.5, -9.4]

In [3]:
get_max(L)

8.2

#### Observações:
* Um dos erros de implementação mais comuns para o cálculo do máximo é inicializar `max_value` com o valor 0. Por que está errado?
* O nome `max` a variável que guarda o valor do máximo parece mais natural do que `max_value`. Mas `max` é palavra-chave em Python e é deselegante usar este tipo de termo em nossos programas. 
* É comum e recomendável nomear funções com verbos e variáveis com substantivos pois, em geral, é o que eles representam no mundo real.

### 2. Mínimo

A função para calcular o mínimo é praticamente igual à para calcular o máximo. Basta inverter o sinal da comparação dentro do laço:

In [4]:
def get_min(L):
    max_value = L[0]
    
    for element in L:
        if min_value > element:
            min_value = element
    
    return min_value

### 3. Média

A média de um conjunto $ X = \{x_1, x_2, ..., x_n\} $ é dada por: $ \sum\limits_{i = 1}^{n} \frac{x_i}{n} $. Assim, para calcular a média de `L` podemos calcular a soma dos valores de `L` e dividí-la por `len(L)`.

Como somar os valores de uma lista tem utilidade por si só, criaremos uma função para fazer isso. Note que, como `max` e `min`, há também uma função `sum` embutida que faz o que queremos.

In [5]:
def get_sum(L):
    value = 0

    for element in L:
        value += element

    return value

In [6]:
get_sum(L)

7.1

Agora, a função que calcula a média só precisa chamar esta função e dividir o valor devolvido pelo número de elementos da lista.

In [7]:
def get_mean(L):
    return get_sum(L) / len(L)

In [8]:
get_mean(L)

0.71

####  Sobre o desempenho da função

Escolhemos calcular a média de `L` fazendo primeiro a soma dos elementos de `L` e depois dividindo-a pelo número de elementos. Uma alternativa seria fazer um laço somando os valores de `L[i]/len(L)` em `value`, como abaixo. Qual deles é preferível?

In [9]:
def get_mean2(L):
    value = 0
    
    for element in L:
        value += element/len(L)
    
    return value

Um modo possível de responder a essa pergunta é: preferimos a implementação que gaste menor tempo para rodar. Para listas de pequenos tamanhos, a implementação não faz muita diferença. Vejamos então a diferença no tempo de execução para o processamento da lista `big_list`, com $ 10^7 $ elementos.

In [10]:
big_list = range(10000000)

In [11]:
result1 = %timeit -o get_mean(big_list)

1 loops, best of 3: 981 ms per loop


In [12]:
result2 = %timeit -o get_mean2(big_list)

1 loops, best of 3: 1.46 s per loop


Vemos que o tempo de execução foi bem maior para a segunda implementação. Isso ocorre porque `get_mean2` faz uma soma, uma divisão, e uma atribuição para cada elemento de `L`, enquanto `get_mean` faz apenas uma soma e uma atribuição. Como essas operações estão dentro de laços grandes, são as operações mais executadas por cada uma das funções. Assim, parece que o número de operações fundamentais que a função `get_mean` executa é aproximadamente $ \frac{2n}{3n} = \frac{2}{3} \approx 0.666... $ daquele que a `get_mean2` executa. Compare com o valor observado entre as melhores execuções de cada uma:

In [13]:
result1.best/result2.best

0.6709513966094954

### 4. Variância

A variância de uma amostra $ X = \{ x_1, x_2, ..., x_n\} $ é definida como $ \frac{1}{n - 1}\sum\limits_{i=1}^{n} (x_i - \bar{X})^2 $, onde $ \bar{X} $ representa a média amostral. Podemos simplificar esta expressão fazendo:
\begin{align}
\frac{1}{n - 1}\sum\limits_{i=1}^{n} (x_i - \bar{X})^2 
&= \frac{1}{n - 1} \left( \sum\limits_{i=1}^{n} (x_i^2 - 2 x_i \bar{X} +  \bar{X}^2) \right) \\
&= \frac{1}{n - 1} \left( \sum\limits_{i=1}^{n} x_i^2 -\sum\limits_{i=1}^{n} 2 x_i \bar{X} + \sum\limits_{i=1}^{n} \bar{X}^2 \right)\\
&= \frac{1}{n - 1} \left( \sum\limits_{i=1}^{n} x_i^2 - 2\bar{X}\sum\limits_{i=1}^{n} x_i + \bar{X}^2 \sum\limits_{i=1}^{n} 1 \right)\\
&= \frac{1}{n - 1} \left( \sum\limits_{i=1}^{n} x_i^2 - 2n\bar{X}^2 + n\bar{X}^2 \right)\\
&= \frac{1}{n - 1} \left( \sum\limits_{i=1}^{n} x_i^2 - n\bar{X}^2 \right)
\end{align}

Como qualquer implementação trivial deve elevar um número ao quadrado, vamos primeiro escrever uma função para fazer isso:

In [14]:
def sqr(x):
    return x*x

Uma possível implementação de uma função que calcula a variância é a seguinte:

In [15]:
def get_variance(L):
    # Estamos supondo que len(L) > 1
    
    sum_sqr = 0
    
    for element in L:
        sum_sqr += element * element
    
    mean = get_mean(L)
    
    return (sum_sqr - len(L) * mean * mean) / (len(L) - 1)

In [16]:
get_variance(L)

34.49877777777778

#### Sobre o desempenho da função:

Como para o cálculo da média, há outras fórmulas em que poderíamos basear nossa implementação. Considere a seguinte implementação baseada na definição:

In [17]:
def get_variance2(L):
    # Estamos supondo que len(L) > 1
    
    value = 0
    
    mean = get_mean(L)
    
    for element in L:
        value += (element - mean)*(element - mean)
    
    return value/(len(L) - 1)

In [18]:
get_variance2(L)

34.49877777777778

Vamos comparar desempenho entre as duas implementações aplicadas sobre nossa lista `big_list`.

In [19]:
result1 = %timeit -o get_variance(big_list)

1 loops, best of 3: 2.33 s per loop


In [20]:
result2 = %timeit -o get_variance2(big_list)

1 loops, best of 3: 3.36 s per loop


Novamente, o tempo de execução foi bem maior para a segunda implementação. Ambas chamam a função `get_mean` que faz aproximadamente $2n$ operações. A função `get_variance2` faz três somas, uma multiplicação, e uma atribuição, $ n $ vezes, resultando num consumo de tempo aproximado de $ 2n + 5n = 7n $. Por sua vez, a `get_variance` faz uma soma, uma multiplicação, e uma atribuição, $ n $ vezes, e seu consumo aproximado é de $ 2n + 3n = 5n $. Isso sugere que o desempenho da função `get_variance` deva ser aproximadamente $ \frac{5n}{7n} = \frac{5}{7} \approx = 0.7143 $ do desempenho da `get_variance2`. Compare com o resultado observado.

In [21]:
result1.best/result2.best

0.6938860194237043

#### Observações:

* Em Python temos alguns modos de elevar um número ao quadrado. Dois deles são:
    1. Usando o operador `**`, e `2**4` resulta em `16`.
    2. Usando a função embutida `pow`, e `pow(2, 4)` resulta em `16`.
    
Não usamos nenhum deles pois, para o caso específico de elevar ao quadrado, os dois são mais lentos que a multiplicação ingênua. Assim, contaminariam os resultados de desempenho da função.

* É muito interessante evitar código repetido nos nossos programas. Considere as funções abaixo:
```python
    def sum_values(L):
        value = 0

        for element in L:
            value += element

        return value

    def sum_sqr_values(L):
        value = 0

        for element in L:
            value += element*element

        return value
```


Imagine que em algum momento, queiramos usar os valores de element/2 e não element em nosso código. Teríamos que mudar os valores nas duas funções, o que traz dificuldades na hora de entender e corrigir o código, podendo levar a erros no programa. Uma alternativa, seria generalizar a função, como abaixo:
```python
    def sum_pow(L, power):
        value = 0

        for element in L:
            value += pow(element, power)

        return value
```

E assim, `sum_values(L) = sum_pow(L, 1)` e `sum_sqr_values(L) = sum_pow(L, 2)`.

### 5. Desvio Padrão

O desvio padrão é definido como a raiz quadrada da variância. Em python, há vários modos de se obter a raiz quadrada de um número `x`. Alguns exemplos são:
* `x**(1/2)`
* `pow(x, 1/2)`
* `math.sqrt(x)`

Porém, neste EP, foi pedido que o aluno implementasse o Método de Newton para o cálculo da raiz quadrada. Uma implementação é 
descrita a seguir.

O Método de Newton usa um algoritmo iterativo que pára quando a diferença absoluta dos valores obtido para iterações consecutivas é menor que um certo erro preestabelecido `SQRT_ERROR`. O valor para este erro no enunciado era de $ 10^{-4} $. Assim:

In [22]:
SQRT_ERROR = 1e-4

In [23]:
SQRT_ERROR


0.0001

Precisamos, também, de uma função para calcular valores absolutos. Sabemos que em Python há uma função embutida para isso chamada `abs`. Porém, é preferível que o aluno implemente uma função equivalente. Abaixo, temos uma implementação para isso.

In [24]:
def get_abs(value):
    if value >= 0:
        return value
    else:
        return -value

Estamos aptos a escrever nossa função que calcula raízes quadradas seguindo o Método de Newton.

In [25]:
def get_sqrt_newton(x):
    r0 = x
    r1 = (x + 1) / 2    # = (r0 + x/r0) / 2

    while get_abs(r1 - r0) >= SQRT_ERROR:
        r0 = r1
        r1 = (r0 + x / r0) / 2

    return r1
    

In [26]:
get_sqrt_newton(10)

3.162277660168379

Compare o resultado obtido com aquele que se obtém usando a função embutida `pow`.

In [27]:
pow(10, 1/2)

3.1622776601683795

Com isso, a implementação da função para calcular o desvio padrão é simplesmente:

In [28]:
def get_sd(L):
    return get_sqrt_newton(get_variance(L))

In [29]:
get_sd(L)

5.873566018849008

#### Observações:

* Se em nosso programa estivermos interessados na variância e no desvio padrão, chamar `get_sd` pode não ser muito esperto. pois ela calcularia a variância novamente. O melhor seria fazer:
```python
    var = get_var(L)
    sd = get_sqrt_newton(var)
```

Escrevi a função `get_sd` desse modo para deixá-la independente de resultados anteriores.